In [2]:
# now a good dataformat to continue working with it a database with entries
# (unique id) timestamp station_id vehicle_id
# this adds some duplication since we store timestamp and station_id more, but now
# we know all the ids and can save only these in the database
import sys, os
import requests
import time
import datetime
import pandas as pd

API_PATH = "https://api.publibike.ch/v1/public"

def merge(path):
    # get stations
    stations = requests.get(API_PATH + "/stations").json()

    # how to read later
    #data2 = pd.read_hdf(DIR + '/data/data.h5', key=str(current_timestamp))
    raw = pd.HDFStore(os.path.join(path, 'data/data.h5'))
    raw_concat = [raw[k] for k in raw.keys()]

    merged = pd.concat(raw_concat)
    return merged

data = merge(os.getcwd())


# access ith frame with raw[raw.keys()[i]]


# What is interesting to analyse
1. Vehicle that change stations
    * by users
    * by publibike
2. Occupation of stations
    * empty stations
3. Ranking of stations
    * How popular are stations
    * How often are popular stations empty but could be even more popular instead

## How to display
* Popularity as heatmap
* Flow of station to station transition (by user or publibke) on flowmap

In [263]:
### get all entries where a vehicle changed stations (station_changes)
### in a second set (station_arrivals) exclude those immediately after a crawl gap, 
### to have actual arrival times +- 5 to 10min accurate
### ATTENTION: this should not be used to track the actual path a vehicle has taken, since some of these
### entries were droped
data.sort_values(by=['vehicle_id', 'timestamp'], inplace=True)

# condition is either different stations or different vehicles to be kept
change_cond = (data.station_id.shift() != data.station_id) | (data.vehicle_id.shift() != data.vehicle_id)

# now require all entries to be within 30 minutes of previous one, to have accurate arrival times
DROP_TIME_THRESHOLD = 30
change_and_uptodate_cond = change_cond & ((data.timestamp-data.timestamp.shift()).fillna(0).astype('timedelta64[m]') < DROP_TIME_THRESHOLD)

# reindex to ensure arrivals and departures have same key set
station_arrivals = data.loc[change_cond].reset_index(drop=True)
station_arrivals_timed = data.loc[change_and_uptodate_cond]

# the departures are always one element before the match in station_id.shift != data.station_id
# problem -> if not equal conditions (with vehicle id jump) then it gets hard to compare to arrivals list
departure_cond = (data.station_id != data.station_id.shift(-1)) | (data.vehicle_id != data.vehicle_id.shift(-1)) # the next id is not the same as the current
station_departures = data.loc[departure_cond].reset_index(drop=True)

# note that a vehicle_id mismatch in departure_cond means the data entry is meaningless, but is kept
# but we find this out once we substract the dep from arr, and can spot the mismatch there

In [230]:
# test = pd.DataFrame({ 'station_id' : [1, 1,1,2,1,1,2,2],
#                       'timestamp' : [pd.Timestamp('20130102'), 
#                                      pd.Timestamp('20130102'),
#                                      pd.Timestamp('20130103'), 
#                                      pd.Timestamp('20130103')] * 2,
#                       'vehicle_id' : [3] * 8})

#(test.timestamp-test.timestamp.shift()).fillna(0).astype('timedelta64[m]') < 30
#test.timestamp.shift() == (test.timestamp)
#cond = (test.station_id.shift() != test.station_id) | (test.vehicle_id.shift() != test.vehicle_id)
#test[cond]
#cond.shift(-1).fillna(False)

(26348, 3)

In [264]:
## count the number of station a vehicle change has occured (new incoming vehicle)
## and the number a specific vehicle has been used
def get_occurence_per_unique_field(pd_field):
    unique_field = pd_field.unique()
    counts = [(pd_field.value_counts()[id], id) for id in unique_field]
    return counts

station_counts = get_occurence_per_unique_field(station_changes.station_id)
vehicle_counts = get_occurence_per_unique_field(station_changes.vehicle_id)


In [266]:
# get the paths all vehicles have taken when a station change occured
subslabels = ['timestamp', 'vehicle_id']

# the first entry in the station_arrivals will be the first reocrded occurence, which is not an arrival 
# but rather the starting point
# the first entry in the departures list corresponds to the first departure, meaning
# that at every index i in the departure the voyage starts until a record in the arrivals at index i+1 occur

# this results in 
# travel time = arrival[i+1] - departure[i] # up to +-2*5min error
travel_time = station_arrivals[subslabels] - station_departures.shift()[subslabels]
# idle time = departure[i] - arrival[i]     # up to +-2*5min error
idle_time = station_departures.timestamp - station_arrivals.timestamp

# for computation on whole array (all vehicle ids) the following problem occus
# whenever a shift between vehicle id occurs, the entry in the travel_time will be meaningless
# we can spot this from also substrcting vehicle ids: non zero entries will be invalid. Therefore
# we can simply set it to NaT, which is then equivalent to the very first entry 
travel_time[travel_time.vehicle_id != 0] = pd.NaT
travel_time = travel_time.timestamp

# Calculation per vehicle_id
# N elements in arrival, N-1 elements in departure
# -> the vehicle was idle at cutoff time, there is not a departure yet
# N elements in arrival, N elements in departure
# -> the vehicle is currently (during cutoff) being moved/used
# note that this is solved by invalidating in between vehicle_id (above) or the cutoff at the shift in the end

# add everything together
movement = pd.concat([station_arrivals, travel_time, idle_time], axis=1)
labels = ['station_id', 'timestamp', 'vehicle_id', 'travel_time', 'idle_time']
movement.columns = labels

movement
#idle_time = station_arrivals_s[['timestamp', 'vehicle_id']] - station_departures_s.shift()[['timestamp', 'vehicle_id']]

,station_id,timestamp,vehicle_id,travel_time,idle_time
0,43,2018-06-05 21:45:02.731332000,3,NaT,0 days 10:45:00.892780
1,48,2018-06-06 08:40:02.904521000,3,0 days 00:09:59.280409,0 days 07:10:00.169627
2,49,2018-06-06 16:00:03.357760000,3,0 days 00:10:00.283612,0 days 01:20:00.460288
3,48,2018-06-06 17:25:03.321320000,3,0 days 00:04:59.503272,0 days 00:14:59.730861
4,47,2018-06-06 18:00:03.277659000,3,0 days 00:20:00.225478,0 days 14:04:59.452133
5,48,2018-06-07 08:20:02.514302000,3,0 days 00:14:59.784510,0 days 21:50:00.512422
6,47,2018-06-14 15:05:02.679728000,3,6 days 08:54:59.653004,0 days 22:40:00.774325
7,49,2018-06-15 14:05:02.842731000,3,0 days 00:19:59.388678,0 days 01:45:00.653737
8,47,2018-06-15 16:10:03.247914000,3,0 days 00:19:59.751446,0 days 22:35:00.462607
9,43,2018-06-16 15:15:03.593159000,3,0 days 00:29:59.882638,0 days 00:49:59.155653


In [267]:
# mini example to figure out stuff above
# dd = pd.DataFrame({ 'station_id' : [1, 1, 2, 2, 3, 3, 4, 4] * 2,
#                       'timestamp' : [pd.Timestamp('20130102'), 
#                                      pd.Timestamp('20130103'),
#                                      pd.Timestamp('20130104'), 
#                                      pd.Timestamp('20130105'),
#                                      pd.Timestamp('20130106'), 
#                                      pd.Timestamp('20130107'),
#                                      pd.Timestamp('20130108'), 
#                                      pd.Timestamp('20130109')] * 2,
#                       'vehicle_id' : [2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3]})


# # get the transition of vehicle_ids right for travel time (with shift)
# m1 = (dd.station_id.shift() != dd.station_id) | (dd.vehicle_id.shift() != dd.vehicle_id)
# m2 = (dd.station_id != dd.station_id.shift(-1)) | (dd.vehicle_id != dd.vehicle_id.shift(-1))

# arr = dd.loc[m1].reset_index(drop=True)
# dep = dd.loc[m2].reset_index(drop=True)

# travel = arr - dep.shift()
# # non matching vehicle ids means its like the first
# # entry, where there is not a matching dep
# travel[travel.vehicle_id != 0] = pd.NaT
# # and only get one column
# travel = travel.timestamp

# idle = dep - arr
# print(idle)